# Step 1: Install and Load FastText
First, install the fasttext library and download a pretrained FastText model if you haven't done so already:

In [ ]:
!pip install fasttext

  Using cached fasttext-0.9.3.tar.gz (73 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
  Using cached numpy-2.1.3-cp313-cp313-win_amd64.whl.metadata (60 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
Using cached numpy-2.1.3-cp313-cp313-win_amd64.whl (12.6 MB)
Failed to build fasttext
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for fasttext (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [119 lines of output]
      C:\Users\MSI\AppData\Local\Temp\pip-build-env-dnrp1lkz\overlay\Lib\site-packages\setuptools\dist.py:495: SetuptoolsDeprecationWarning: Invalid dash-separated options
      !!
      
              ********************************************************************************
              Usage of dash-separated 'description-file' will not be supported in future
              versions. Please use the underscore name 'description_file' instead.
      
              By 2025-Mar-03, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
              ********************************************************************************
      
      !!
        o

Load the pretrained FastText model:

In [4]:
import fasttext
import fasttext.util

# Load the pretrained FastText English embeddings
# fasttext.util.download_model('en', if_exists='ignore')  # Replace 'en' with 'vi' for Vietnamese
ft = fasttext.load_model('cc.en.300.bin')  # Load 300-dimensional FastText English embeddings

ModuleNotFoundError: No module named 'fasttext'

# Step 2: Modify the load_data and Embedding Extraction
Replace the BERT tokenization and embeddings in the Question 1 with FastText embeddings for each word in the sentences.

In [ ]:
import fasttext
import fasttext.util

# Load the FastText model for Vietnamese
# fasttext.util.download_model('en', if_exists='ignore')  # Only runs if the model is not already downloaded
# ft = fasttext.load_model('cc.en.300.bin')  # Adjust the path if needed

# Define the function to load data
def load_data(filepath):
    sentences, labels = [], []
    with open(filepath, 'r') as f:
        sentence, label = [], []
        for line in f:
            if line.startswith('-DOCSTART-') or line == "\n":
                if sentence:
                    sentences.append(sentence)
                    labels.append(label)
                sentence, label = [], []
            else:
                parts = line.strip().split()
                if len(parts) == 4:  # Only process lines with exactly four parts
                    word, _, _, tag = parts
                    sentence.append(word)
                    label.append(tag)
        if sentence:
            sentences.append(sentence)
            labels.append(label)
    return sentences, labels

# Load sentences and labels as before
train_sentences, train_labels = load_data('/content/drive/MyDrive/Colab_Notebooks/MIDTERM_NLP/train.txt')
test_sentences, test_labels = load_data('/content/drive/MyDrive/Colab_Notebooks/MIDTERM_NLP/test.txt')
valid_sentences, valid_labels = load_data('/content/drive/MyDrive/Colab_Notebooks/MIDTERM_NLP/valid.txt')

# Convert sentences to FastText embeddings
def get_fasttext_embeddings(sentences):
    embedded_sentences = []
    for sentence in sentences:
        sentence_embeddings = [ft.get_word_vector(word) for word in sentence]
        embedded_sentences.append(sentence_embeddings)
    return embedded_sentences

# Get embeddings for each dataset
train_embeddings = get_fasttext_embeddings(train_sentences)
test_embeddings = get_fasttext_embeddings(test_sentences)
valid_embeddings = get_fasttext_embeddings(valid_sentences)


# Step 3: Update the NERDataset Class for FastText Embeddings
Since FastText embeddings are already in vector form, you won’t need additional token offsets or padding. Each word vector can be directly used as input to the model.

In [ ]:
import fasttext
import numpy as np

# Tải mô hình FastText đã huấn luyện trước cho tiếng Việt
fasttext_model = fasttext.load_model('cc.en.300.bin')  # Đường dẫn đến mô hình tiếng Việt đã huấn luyện

# Hàm chuyển đổi câu thành vector embedding với FastText
def sentence_to_embedding(sentences):
    embedded_sentences = []
    for sentence in sentences:
        embedded_sentence = [fasttext_model.get_word_vector(word) for word in sentence]
        embedded_sentences.append(embedded_sentence)
    return embedded_sentences

# Áp dụng FastText cho các câu train, test, valid
train_embeddings = sentence_to_embedding(train_sentences)
test_embeddings = sentence_to_embedding(test_sentences)
valid_embeddings = sentence_to_embedding(valid_sentences)

# Chuyển đổi các embedding và nhãn thành dạng PyTorch Tensor Dataset
import torch

class FastTextNERDataset(torch.utils.data.Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.embeddings[idx], dtype=torch.float32),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

    def __len__(self):
        return len(self.labels)

# Tạo tập dữ liệu huấn luyện, kiểm thử và xác thực
train_dataset = FastTextNERDataset(train_embeddings, train_labels)
test_dataset = FastTextNERDataset(test_embeddings, test_labels)
valid_dataset = FastTextNERDataset(valid_embeddings, valid_labels)

# Step 4: Train the Model with Hugging Face Trainer
Since we’re using a custom model, we can train it directly with the Hugging Face Trainer:

In [ ]:
# Define training arguments
# Các tham số huấn luyện được định nghĩa bằng TrainingArguments:
training_args = TrainingArguments(
    # Thư mục lưu trữ kết quả huấn luyện
    output_dir='./results',
    # evaluation_strategy="epoch" chỉ định việc đánh giá vào cuối mỗi epoch
    #(một lần duyệt qua toàn bộ dữ liệu huấn luyện).
    evaluation_strategy="epoch",
    # learning_rate=2e-5: Tốc độ học của mô hình, xác định mức độ điều chỉnh trọng số sau mỗi lần cập nhật.
    learning_rate=2e-5,
    # Kích thước batch cho mỗi thiết bị trong quá trình huấn luyện.
    per_device_train_batch_size=16,
    # Kích thước batch cho mỗi thiết bị trong quá trình đánh giá.
    per_device_eval_batch_size=16,
    # Số lượng epoch để huấn luyện mô hình.
    num_train_epochs=3,
    # Hệ số giảm trọng số, giúp tránh overfitting bằng cách giảm giá trị của các trọng số lớn.
    weight_decay=0.01,
)

# Define Trainer
# Khởi tạo một đối tượng Trainer từ thư viện Transformers của Hugging Face
#Trainer một lớp giúp bạn dễ dàng huấn luyện và đánh giá mô hình.
trainer = Trainer(
    # model=model: Đây là mô hình bạn muốn huấn luyện.
    #Nó có thể là bất kỳ mô hình nào từ thư viện Transformers, chẳng hạn như BERT, GPT-2, v.v.
    model=model,
    # Đây là các tham số huấn luyện mà đã được định nghĩa ở trên,
    #chẳng hạn như tốc độ học, số lượng epoch, kích thước batch, v.v.
    args=training_args,
    # train_dataset=train_dataset là tập dữ liệu huấn luyện dùng để huấn luyện mô hình.
    train_dataset=train_dataset,
    # eval_dataset=test_dataset là tập dữ liệu đánh giá dùng để đánh giá mô hình trong quá trình huấn luyện.
    eval_dataset=test_dataset
)

# Sử dụng method train() của trainer để train model.
# Sau khi train xong, model sẽ sẵn sàng để đánh giá và sử dụng cho các tác vụ dự đoán.
trainer.train()

# Step 5: Evaluation

In [ ]:
import numpy as np

# Evaluate the model on the validation dataset
# Mô hình được đánh giá trên tập valid_dataset bằng cách sử dụng trainer.evaluate().
#Nó trả về các chỉ số như độ chính xác và độ mất mát trên valid_dataset.
# evaluation_results = trainer.evaluate(eval_dataset=valid_dataset)
# in ra {'eval_loss': 0.021987076848745346,
#'eval_runtime': 25.482,
#'eval_samples_per_second': 127.541,
#'eval_steps_per_second': 8.006,
#'epoch': 3.0}
# print(evaluation_results)

# Get predictions on the test set
# method trainer.predict() được sử dụng để lấy các dự đoán trên tập test_dataset
#labels là ID nhãn thực từ tập test_dataset.
predictions, labels, _ = trainer.predict(test_dataset)
# predictions chứa các điểm số thô cho mỗi lớp,
#được chuyển đổi thành ID nhãn bằng cách lấy argmax.
predictions = np.argmax(predictions, axis=2)

# Define a function to convert the label IDs back to label names
def convert_labels(predictions, labels, label_list):
    pred_list = []
    true_list = []
    for pred, label in zip(predictions, labels):
        pred_list.append([label_list[p] for p in pred])
        true_list.append([label_list[l] for l in label])
    return pred_list, true_list

# Convert the predictions and labels back to their original format
pred_labels, true_labels = convert_labels(predictions, test_labels, label_list)
# Generate a classification report
from sklearn.metrics import classification_report
# Flatten the lists for easier comparison
true_labels_flat = [item for sublist in true_labels for item in sublist]
pred_labels_flat = [item for sublist in pred_labels for item in sublist]
# Generate and print the classification report
print(classification_report(true_labels_flat, pred_labels_flat, target_names=label_list))